# Fine-tune LLaMA‑3.1 8B (QLoRA) trên Google Colab (T4)

**Phiên bản:** file hướng dẫn đầy đủ, tối ưu cho GPU T4 (16 GB VRAM).

**Ghi chú quan trọng:**
- LLaMA‑8B yêu cầu cẩn trọng trên T4. Script này sử dụng QLoRA, bitsandbytes và offload để giảm VRAM.
- Bạn *nên* đổi sang A100 nếu có khối lượng dataset lớn (>10k) hoặc muốn huấn luyện nhanh hơn.

---


In [ ]:
# 1) Cài đặt thư viện cần thiết
# Phiên bản được chọn tương thích với QLoRA + TRL + PEFT
!pip install -q -U "transformers>=4.40.0" accelerate bitsandbytes datasets peft trl sentencepiece safetensors huggingface_hub

# Kiểm tra GPU
import torch, os
print('Torch version:', torch.__version__)
if torch.cuda.is_available():
    print('CUDA device:', torch.cuda.get_device_name(0))
    try:
        print('CUDA memory (GB):', round(torch.cuda.get_device_properties(0).total_memory/1024**3,1))
    except:
        pass
else:
    print('CUDA not available - Colab runtime must have GPU enabled (Runtime > Change runtime type > GPU)')

## 2) Đăng nhập HuggingFace

Bạn cần HuggingFace token để tải model `meta-llama/Llama-3.1-8b-instruct`. Tạo token tại: https://huggingface.co/settings/tokens

Chạy cell dưới và dán token khi được hỏi.

In [ ]:
from huggingface_hub import login
# Chạy lệnh và dán token vào prompt
login()

## 3) Chuẩn bị dataset

- Nếu bạn chưa có dataset, cell bên dưới sẽ tạo một **dataset mẫu nhỏ** (100 mẫu) để thử nghiệm pipeline.
- Khi bạn có dataset thật, upload file `train.jsonl` vào `/content/data/train.jsonl` (mỗi dòng một JSON: `{"instruction":..., "input":..., "output":...}`)


In [ ]:
# Tạo folder data và dataset mẫu 100 mẫu để test
import os, json, random
os.makedirs('/content/data', exist_ok=True)
def make_sample(i):
    instr = "Phân tích kết quả học tập học kỳ 1 và viết nhận xét ngắn 3 câu."
    sample_input = {"Toan": round(random.uniform(4,10),1),
                    "Ly": round(random.uniform(3,10),1),
                    "Van": round(random.uniform(3,10),1),
                    "Anh": round(random.uniform(3,10),1)}
    out = f"Học sinh có điểm Toán {sample_input['Toan']}, Lý {sample_input['Ly']}, Văn {sample_input['Van']}, Anh {sample_input['Anh']}. Nhận xét: ..."
    return {"instruction": instr, "input": json.dumps(sample_input, ensure_ascii=False), "output": out}

with open('/content/data/train.jsonl','w', encoding='utf-8') as f:
    for i in range(100):
        f.write(json.dumps(make_sample(i), ensure_ascii=False) + '\n')

print('Tạo file /content/data/train.jsonl (100 mẫu) - Replace bằng dataset thật khi có.')

## 4) Load model & tokenizer (QLoRA-friendly)

**Chú ý:** model tên `meta-llama/Llama-3.1-8b-instruct` được sử dụng. Cell này thiết lập load 4bit và device_map thích hợp với Colab T4.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "meta-llama/Llama-3.1-8b-instruct"

print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)

print('Loading model in 4-bit (may take a few minutes)...')
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)
print('Done loading model (4-bit).')

## 5) Cấu hình QLoRA (PEFT/LoRA)

Tối ưu cấu hình LoRA dành cho T4: sử dụng r nhỏ, dropout vừa phải. Bạn có thể điều chỉnh `r` hoặc `lora_alpha` nếu cần.

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

print('Applying LoRA...')
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 6) Chuẩn bị dataset cho Trainer

Sử dụng `datasets` để load file JSONL đã upload (hoặc mẫu). Chúng ta sẽ tạo trường `text` chứa prompt đầy đủ (Instruction + Input + Response).

In [ ]:
from datasets import load_dataset

data_path = '/content/data/train.jsonl'
ds = load_dataset('json', data_files=data_path)['train']

def to_text(example):
    instr = example.get('instruction','')
    inp = example.get('input','')
    out = example.get('output','')
    if inp:
        txt = f"### Instruction:\n{instr}\n\n### Input:\n{inp}\n\n### Response:\n{out}"
    else:
        txt = f"### Instruction:\n{instr}\n\n### Response:\n{out}"
    return {'text': txt}

ds = ds.map(to_text)
print('Dataset size:', len(ds))
print(ds[0])

## 7) Cấu hình Trainer & Huấn luyện

Thiết lập phù hợp cho T4: very small per-device batch size + gradient accumulation. Điều chỉnh `num_train_epochs` theo dataset của bạn.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir='./llama_8b_qlora_output',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    optim='paged_adamw_8bit',
    remove_unused_columns=False
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    dataset_text_field='text',
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False
)

print('Ready to train. To start training, run: trainer.train()')

In [ ]:
# 8) BẮT ĐẦU HUẤN LUYỆN
# Bật cell này nếu bạn đã kiểm tra mọi thứ và muốn bắt đầu training.
# LƯU Ý: training có thể tốn thời gian; với dataset mẫu 100 mẫu, nó sẽ hoàn thành nhanh.
# Nếu runtime bị out-of-memory, giảm gradient_accumulation_steps hoặc max_seq_length.

# trainer.train()

## 9) Lưu LoRA adapters & tokenizer

In [ ]:
# Sau khi huấn luyện xong, lưu LoRA adapters và tokenizer
out_dir = '/content/lora-output'
model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)
print('Saved LoRA adapters to', out_dir)

## 10) (Tùy chọn) Merge LoRA thành model hoàn chỉnh

Nếu muốn xuất model hoàn chỉnh để deploy, bạn có thể merge adapters (chú ý VRAM lớn khi merge).

In [ ]:
# from peft import AutoPeftModelForCausalLM
# merged = AutoPeftModelForCausalLM.from_pretrained(out_dir, device_map='auto')
# merged = merged.merge_and_unload()
# merged.save_pretrained('/content/llama-8b-merged')

## 11) Kiểm tra inference (sau khi lưu LoRA)


In [ ]:
# Kiểm tra inference nhanh (nếu LoRA đã được load)
from transformers import pipeline
try:
    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map='auto')
    prompt = "### Instruction:\nPhân tích ngắn kết quả học kỳ cho học sinh: Toán 8.7, Lý 5, Văn 7.\n\n### Response:\n"
    out = gen(prompt, max_new_tokens=200, do_sample=False)[0]['generated_text']
    print(out)
except Exception as e:
    print('Inference test failed:', e)
    print('If inference fails on-colab, try loading the LoRA adapters via the saved directory and re-run.')

----

# GHI CHÚ CUỐI
- Thay `data/train.jsonl` bằng dataset thật của bạn (3.000–5.000 mẫu tối thiểu để bắt đầu).
- Nếu gặp OOM: giảm `gradient_accumulation_steps`, `max_seq_length`, hoặc chuyển sang GPU lớn hơn (A100).
- Muốn mình tạo dataset synthetic 3.000 mẫu chuẩn instruct? Trả lời 'Có' và mình sẽ tạo luôn file dataset cho bạn.

Chúc bạn thành công!